## Load libraries and model


Firt step: Load libraries


In [ ]:
import torch, polars as pl
from gnnepcsaft.train.utils import build_train_dataset, build_test_dataset, calc_deg
from gnnepcsaft.configs.default import get_config
from gnnepcsaft.train.models import GNNePCSAFTL, create_model
import numpy as np
from torch_geometric.loader import DataLoader
from rdkit import Chem
from gnnepcsaft.demo.utils import (
    plotdata,
    model_para_fn,
    datacsv,
    plotparams,
    save_exported_program,
)

torch.cuda.is_available()

Second step: Load data


In [ ]:
es = build_train_dataset("gnnepcsaft", "esper")
tml_val, tml_train, tml_assoc = build_test_dataset("gnnepcsaft", es)
es_loader = DataLoader(es, batch_size=512, shuffle=True)
tml_loader = DataLoader(tml_train, batch_size=len(tml_train), shuffle=False)


device = torch.device("cpu")

Third Step: Define and load models


In [ ]:
model = GNNePCSAFTL.load_from_checkpoint(
    "gnnepcsaft/train/checkpoints/esper_msigmae_5.4-epoch=49999-train_mape=0.0046.ckpt",
    "cpu",
)
model.eval()

## Evaluations


Run model testing on data


In [ ]:
model1_para, model1_array = model_para_fn(model3.model, model1.model.eval())
model2_para, model2_array = model_para_fn(model3.model, model2.model.eval())

Save test results to csv file


In [ ]:
data2 = datacsv(model2_para)
data1 = datacsv(model1_para)
model1data = pl.DataFrame(data1)
model2data = pl.DataFrame(data2)
model1data.write_csv("../model1.csv")
model2data.write_csv("../model2.csv")

## plotting


In [ ]:
inchi = input("InChI: ")
molecule_name = "UNKNOWN"
plotdata(
    inchi,
    molecule_name,
    [model.model],
    model.model,
)

In [ ]:
mol = Chem.MolFromInchi(inchi, removeHs=False, sanitize=True)
n_pyrimidine = len(mol.GetSubstructMatches(Chem.MolFromSmiles("O")))
print(Chem.MolToSmiles(mol, isomericSmiles=True))

mol

In [ ]:
smiles = ["C" * i + "(=O)OCC" for i in range(1, 100)]

In [ ]:
smiles = ["C" * i for i in range(1, 100)]

In [ ]:
smiles = ["C" * i + "O" for i in range(1, 100)]

In [ ]:
plotparams(smiles, [model.model], "UNKNOWN")

In [ ]:
test_inchis = []
for gh in tml_assoc:
    test_inchis.append(gh.InChI)
inchis = []
for smile in smiles:
    mol = Chem.MolFromSmiles(smile)
    inchi = Chem.MolToInchi(mol)
    if inchi in test_inchis:
        inchis.append(inchi)

In [ ]:
inchis